In [1]:
import pandas as pd
from __future__ import print_function
import keras
from keras.models import Sequential, Model, Input
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import LSTM
import sklearn
from sklearn import preprocessing, model_selection
import tensorflow as tf
from scipy import stats
import matplotlib.pyplot as plt
import PIL
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, cross_val_score
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

accept_df = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', compression='gzip')
reject_df = pd.read_csv('rejected_2007_to_2018Q4.csv.gz', compression='gzip')

In [3]:
# =============================================================================
# Feature selection, renaming and target variable definition
# =============================================================================

accept_df = accept_df.dropna(subset=['fico_range_low']) 
reject_df['Debt-To-Income Ratio'] = reject_df['Debt-To-Income Ratio'].str.replace('%', '')
reject_df['Debt-To-Income Ratio']= reject_df['Debt-To-Income Ratio'].astype(float)

accept_df_new = accept_df[[
        'emp_length', 'dti','loan_amnt','fico_range_low']]
accept_df_new['approved'] = 1

reject_df_new = reject_df[[
        'Employment Length', 'Debt-To-Income Ratio','Amount Requested','Risk_Score']]
reject_df_new['approved'] = 0

#Renaming columns
reject_df_new = reject_df_new.rename(index=str, columns = {'Zip Code': 'zip_code',
                                      'State':'addr_state',
                                      'Employment Length':'emp_length', 
                                      'Debt-To-Income Ratio':'dti',
                                      'Amount Requested':'loan_amnt',
                                      'Risk_Score':'fico',
                                       'Loan Title': 'title'})

accept_df_new = accept_df_new.rename(index=str, columns = {'fico_range_low': 'fico'})

In [4]:
# =============================================================================
# Merging accepted and rejected applications databases
# =============================================================================

approve_df=reject_df_new.append(accept_df_new)

In [6]:
# =============================================================================
# Data cleaning
# =============================================================================

years = ['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years',
        '9 years', '10+ years']
years_num = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for i in range(len(years)):
    approve_df['emp_length'] = approve_df['emp_length'].replace(years[i], years_num[i])

approve_df['fico']=approve_df['fico'].where(approve_df['fico']>=0,0)
approve_df['dti']=approve_df['dti'].where(approve_df['dti']>=0,approve_df['dti'].mean())
approve_df['loan_amnt']=approve_df['loan_amnt'].where(approve_df['loan_amnt']>=0,approve_df['loan_amnt'].mean())
approve_df['emp_length']=approve_df['emp_length'].where(approve_df['emp_length']>=0,approve_df['emp_length'].mean())

In [7]:
# =============================================================================
# Separating features and target variable
# =============================================================================

X = approve_df.drop('approved', axis=1)
y = approve_df['approved']

In [8]:
# =============================================================================
# Data scaling
# =============================================================================

cont_vars=['emp_length','dti','loan_amnt','fico']
sc = StandardScaler()
X[cont_vars]=sc.fit_transform(X[cont_vars])

with open('scaler_final.pkl', 'wb') as f:
        pickle.dump(sc, f)

In [10]:
# =============================================================================
# Separating the database into training and testing sets
# =============================================================================

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.90, random_state=42) 

In [18]:
# =============================================================================
# Fully connected neural network with two hidden relu layers and a sigmoid
# activation function
# =============================================================================

model = Sequential()
model.add(Dense(40, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [39]:
# =============================================================================
# Model fitting
# =============================================================================

model.fit(X_train, y_train, epochs=200, batch_size=1000)

Epoch 1/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0691 - acc: 0.9735
Epoch 2/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9735
Epoch 3/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9736
Epoch 4/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9736
Epoch 5/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0691 - acc: 0.9736
Epoch 6/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9736
Epoch 7/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9736
Epoch 8/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0690 - acc: 0.9736
Epoch 9/50
2990940/2990940 [==============================] - 7s 2us/step - loss: 0.0691 - acc: 0.9735
Epoch 10/50
2990940/2990940 [==============================] - 7s 2us/ste

In [40]:
# =============================================================================
# Model saving
# =============================================================================

with open('NN_model_final.pkl', 'wb') as f:
        pickle.dump(model, f)

In [42]:
# =============================================================================
# Prediction generating
# =============================================================================

y_predicted_test = model_NN.predict(X_test)

In [43]:
# =============================================================================
# Calculating the confusion matrix
# =============================================================================

tn, fp, fn, tp=confusion_matrix(y_test, np.round(y_predicted_test,0)).ravel()
print('TN:',tn, 'FP:',fp, 'FN:',fn, 'TP:',tp)

TN: 24437118 FP: 447034 FN: 210861 TP: 1823456


In [46]:
# =============================================================================
# Error rate
# =============================================================================

(fp + fn)/(tn + fp + fn + tp)*100

2.4440282989348314

In [2]:
# =============================================================================
# False positive rate
# =============================================================================

(fp)/(fp + tn)*100

1.7964606549582243

In [1]:
# =============================================================================
# False negative rate
# =============================================================================

(fn)/(fn + tp)*100

10.365198737463237